In [1]:
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis.compute_dp_sgd_privacy_lib import compute_dp_sgd_privacy

ImportError: cannot import name 'estimator_training' from 'tensorflow.python.distribute' (/Users/sohammandal/Developer/mlops-summer-2025/venv/lib/python3.11/site-packages/tensorflow/python/distribute/__init__.py)

In [1]:
import tensorflow as tf
import tensorflow_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

ImportError: cannot import name 'estimator_training' from 'tensorflow.python.distribute' (/Users/sohammandal/Developer/mlops-summer-2025/venv/lib/python3.11/site-packages/tensorflow/python/distribute/__init__.py)

In [ ]:
v2_train = pd.read_csv("Data Versioning/v2_train.csv")
v2_test = pd.read_csv("Data Versioning/v2_test.csv")

features = ['age', 'height', 'weight']
target = 'total_lift'

v2_train_filtered = v2_train[features + [target]].dropna()
X_train = v2_train_filtered[features].values.astype(np.float32)
y_train = v2_train_filtered[target].values.astype(np.float32).reshape(-1, 1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:


# Build a simple feedforward model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# DP optimizer parameters
l2_norm_clip = 1.5
noise_multiplier = 1.1
num_microbatches = 32
learning_rate = 0.05
batch_size = 128
epochs = 10

# Use DP optimizer
optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)

# Define loss
loss = tf.keras.losses.MeanSquaredError(reduction=tf.losses.Reduction.NONE)

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['mae'])

# Train the DP model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# ---- Compute DP guarantee ----
n = X_train.shape[0]
delta = 1 / n
epsilon, _ = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=n,
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=epochs,
    delta=delta
)

print(f"\nDP model trained with ε = {epsilon:.2f}, δ = {delta:.1e}")
